In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
from datasets.LMDB.caffedb import *
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from skimage.transform import warp
from affine_transformer.numpy.interpolation import *

In [ ]:
# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False,
                           transform=transforms.ToTensor())

canvas = [42, 42]

train_data = train_dataset.train_data.numpy()
train_data_transformed = np.zeros((train_data.shape[0], canvas[0], canvas[1]))
train_data_center_only = np.zeros((train_data.shape[0], canvas[0], canvas[1]))
train_data_rand_placed = np.zeros((train_data.shape[0], canvas[0], canvas[1]))

In [ ]:
for i in range(train_data.shape[0]):

    img = train_data[i]

    plt.figure(figsize=(7, 7))
    plt.imshow(img.reshape(28, 28), cmap='gray', interpolation='none')
    plt.title('Original MNIST', fontsize=20)
    plt.axis('off')
    plt.show()

    rotation = np.random.uniform(-np.pi/4, np.pi/4) # * 180 / np.pi
    # scale = np.random.uniform(0.7, 1.2)
    # scale = np.random.uniform(0.7, 10/3)
    scale = np.random.uniform(0.7, 0.7)
    # scale = np.random.uniform(0.83, 1.42) # corresponding to STN (1/0.7, 1/1.2)

    R = np.array([[np.cos(rotation), -np.sin(rotation), 0],
                  [np.sin(rotation),  np.cos(rotation), 0],
                  [      0,                  0,         1]])
    S = np.array([[scale, 0,  0],
                  [0, scale,  0],
                  [0,   0,    1]])

    T = (np.dot(S, R))[0:2,:]

    im = affine_transformer(img[None, :,:, None], S)
    S2 = np.array([[1, 0,  0],
                  [0, 1,  0],
                  [0,   0,    1]])
    im2 = affine_transformer(img[None, :, :, None], S2, 40, 40)
    im2 = im2[:,6:34,6:34,:]
    # train_data[i,:,:] = rotate(train_data[i], rotation, reshape=False)

    # plt.figure(figsize=(7, 7))
    # plt.imshow(im[0,:,:,0].reshape(28, 28), cmap='gray', interpolation='none')
    # plt.title('Transformed MNIST', fontsize=20)
    # plt.axis('off')
    # plt.show()
    #
    # plt.figure(figsize=(7, 7))
    # plt.imshow(im2[0,:,:,0].reshape(28, 28), cmap='gray', interpolation='none')
    # plt.title('Transformed MNIST', fontsize=20)
    # plt.axis('off')
    # plt.show()


    placed_h_random = np.random.randint(0 - 0.1, canvas[0] - 28)
    placed_w_random = np.random.randint(0 - 0.1, canvas[1] - 28)
    placed_h_center = (canvas[0] - 28) / 2
    placed_w_center = (canvas[1] - 28) / 2

    train_data_transformed[i, placed_h_center:placed_h_center + 28, placed_w_center:placed_w_center + 28] = im[0,:,:,0]
    train_data_center_only[i, placed_h_center:placed_h_center + 28, placed_w_center:placed_w_center + 28] = img[:,:]
    train_data_rand_placed[i, placed_h_random:placed_h_random + 28, placed_w_random:placed_w_random + 28] = img[:,:]

    test = train_data_center_only[i]
    test = affine_transformer(test[None, :, :, None], S2, 60, 60)
    test = test[:,9:51,9:51,:]


    plt.figure(figsize=(7, 7))
    plt.imshow(train_data_transformed[i].reshape(canvas[0], canvas[1]), cmap='gray', interpolation='none')
    plt.title('trans', fontsize=20)
    plt.axis('off')
    plt.show()

    plt.figure(figsize=(7, 7))
    plt.imshow(test.reshape(canvas[0], canvas[1]), cmap='gray', interpolation='none')
    plt.title('scale and crop', fontsize=20)
    plt.axis('off')
    plt.show()

    plt.figure(figsize=(7, 7))
    plt.imshow(train_data_center_only[i].reshape(canvas[0], canvas[1]), cmap='gray', interpolation='none')
    plt.title('no scale', fontsize=20)
    plt.axis('off')
    plt.show()